In [1]:
# !pip install pyarrow

In [2]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle

#import xgbtune 

In [3]:
df = pd.read_parquet("../data/dataset.parquet")
df

,date,open,high,low,close,adj close,volume,up_down_close_1,volitility_open_7,volitility_pct_change_7_open,...,close_percent_change_1_lag_10,close_percent_change_1_lag_14,close_percent_change_1_lag_21,close_percent_change_1_lag_30,close_percent_change_1_lag_60,day_of_week,day_of_month,month,year,week_of_year
0,2004-08-10,0.542679,0.563214,0.541964,0.562857,0.475869,351036000,-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,10,8,2004,33
1,2004-08-11,0.555357,0.555893,0.540357,0.553750,0.468169,322392000,-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2,11,8,2004,33
2,2004-08-12,0.543750,0.550893,0.540714,0.542321,0.458507,226200800,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3,12,8,2004,33
3,2004-08-13,0.546429,0.558571,0.542857,0.550714,0.465603,328048000,-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4,13,8,2004,33
4,2004-08-16,0.553571,0.566429,0.547143,0.549643,0.464697,435674400,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,16,8,2004,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999,2024-06-20,213.929993,214.240005,208.850006,209.679993,209.679993,86172500,-1,8.093688,0.032104,...,-0.007097,0.005019,0.006857,0.001864,-0.006673,3,20,6,2024,25
5000,2024-06-21,210.389999,211.889999,207.110001,207.490005,207.490005,246421400,1,3.274162,0.032110,...,0.012392,0.009259,-0.007538,0.010014,0.021213,4,21,6,2024,25
5001,2024-06-24,207.720001,212.699997,206.589996,208.139999,208.139999,80727000,1,3.174396,0.019971,...,-0.019148,0.001649,-0.021058,-0.008235,-0.010559,0,24,6,2024,26
5002,2024-06-25,209.149994,211.380005,208.610001,209.070007,209.070007,56713900,1,3.384621,0.013439,...,0.072649,0.007821,0.016588,0.017645,-0.008456,1,25,6,2024,26


In [4]:
# target = "target_close"
target = "close_percent_change_1"

x = df.drop(columns=[target]).set_index("date")
y = df[["date", target]].set_index("date")
y

,close_percent_change_1
date,
2004-08-10,-0.016180
2004-08-11,-0.020639
2004-08-12,0.015476
2004-08-13,-0.001945
2004-08-16,0.002924
...,...
2024-06-20,-0.010444
2024-06-21,0.003133
2024-06-24,0.004468


In [5]:
x

,open,high,low,close,adj close,volume,up_down_close_1,volitility_open_7,volitility_pct_change_7_open,high_low_7,...,close_percent_change_1_lag_10,close_percent_change_1_lag_14,close_percent_change_1_lag_21,close_percent_change_1_lag_30,close_percent_change_1_lag_60,day_of_week,day_of_month,month,year,week_of_year
date,,,,,,,,,,,,,,,,,,,,,
2004-08-10,0.542679,0.563214,0.541964,0.562857,0.475869,351036000,-1,NaN,NaN,0.021250,...,NaN,NaN,NaN,NaN,NaN,1,10,8,2004,33
2004-08-11,0.555357,0.555893,0.540357,0.553750,0.468169,322392000,-1,NaN,NaN,0.015536,...,NaN,NaN,NaN,NaN,NaN,2,11,8,2004,33
2004-08-12,0.543750,0.550893,0.540714,0.542321,0.458507,226200800,1,NaN,NaN,0.010179,...,NaN,NaN,NaN,NaN,NaN,3,12,8,2004,33
2004-08-13,0.546429,0.558571,0.542857,0.550714,0.465603,328048000,-1,NaN,NaN,0.015714,...,NaN,NaN,NaN,NaN,NaN,4,13,8,2004,33
2004-08-16,0.553571,0.566429,0.547143,0.549643,0.464697,435674400,1,NaN,NaN,0.019286,...,NaN,NaN,NaN,NaN,NaN,0,16,8,2004,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-20,213.929993,214.240005,208.850006,209.679993,209.679993,86172500,-1,8.093688,0.032104,5.389999,...,-0.007097,0.005019,0.006857,0.001864,-0.006673,3,20,6,2024,25
2024-06-21,210.389999,211.889999,207.110001,207.490005,207.490005,246421400,1,3.274162,0.032110,4.779999,...,0.012392,0.009259,-0.007538,0.010014,0.021213,4,21,6,2024,25
2024-06-24,207.720001,212.699997,206.589996,208.139999,208.139999,80727000,1,3.174396,0.019971,6.110001,...,-0.019148,0.001649,-0.021058,-0.008235,-0.010559,0,24,6,2024,26


In [6]:
table_info = pd.DataFrame()

# Print columns that are not int, float, bool or category
for col in x.columns:
    dtype = x[col].dtype
    if dtype not in ["int", "float64", "bool", "category"]:
        table_info = pd.concat([table_info, pd.DataFrame({"Column Name": [col], "Data Type": [x[col].dtype]})])

table_info

,Column Name,Data Type
0,volume,int64
0,on_balance_volume_1,int64
0,dividends_event_name,object
0,splits_event_name,object


In [7]:
# Convert columns to float if they are not int, float, bool or category. Handle Cannot cast DatetimeArray to dtype float64 (XGBoosted models cannot use strings, but categories as enumerated values)
for col in x.columns:
    dtype = x[col].dtype
    if dtype not in ["int", "float64", "bool", "category"]:
        try:
            x[col] = x[col].astype("float")
        except:
            # drop datetime columns
            x = x.drop(columns=[col])

            print(f"Dropped Column: {col}")

Dropped Column: dividends_event_name
Dropped Column: splits_event_name


In [8]:
# # Normalizing the features between 0 and 1
# y_scaler = MinMaxScaler()
# y = y_scaler.fit_transform(y.values.reshape(-1, 1)).flatten()

# x_scaler = MinMaxScaler()
# x = x_scaler.fit_transform(x)

# y

In [9]:
#    All Features   | Target
# +-----------------+---------+
# | x_train         | y_train | <- 85% of the data which is used for training
# |                 |         |
# +-----------------+---------+
# | x_test          | y_test  | <- 15% of the data which is used for testing
# +-----------------+---------+


# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, shuffle=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)
# x_train

In [10]:
# y_test

In [11]:
x_train

,open,high,low,close,adj close,volume,up_down_close_1,volitility_open_7,volitility_pct_change_7_open,high_low_7,...,close_percent_change_1_lag_10,close_percent_change_1_lag_14,close_percent_change_1_lag_21,close_percent_change_1_lag_30,close_percent_change_1_lag_60,day_of_week,day_of_month,month,year,week_of_year
date,,,,,,,,,,,,,,,,,,,,,
2023-01-18,136.820007,138.610001,135.029999,135.210007,134.105194,69672800.0,1,2.460460,0.016204,3.580002,...,0.010314,-0.030685,-0.014579,-0.007983,0.027059,2,18,1,2023,3
2017-10-17,39.945000,40.217499,39.807499,40.117500,37.700481,75989200.0,-1,0.358985,0.004225,0.410000,...,-0.006473,-0.006160,0.000378,-0.001049,0.004274,1,17,10,2017,42
2016-09-23,28.605000,28.697500,27.887501,28.177500,26.020861,209924800.0,1,0.189031,0.019884,0.809999,...,0.022399,-0.000279,-0.004258,0.002316,0.012712,4,23,9,2016,38
2020-09-22,112.680000,112.860001,109.160004,111.809998,109.392204,183055400.0,-1,4.498105,0.046312,3.699997,...,0.039887,-0.020718,0.011960,-0.029740,0.023047,1,22,9,2020,39
2016-10-12,29.337500,29.495001,29.187500,29.334999,27.089764,150347200.0,-1,0.470730,0.008252,0.307501,...,-0.015533,-0.016664,0.035387,0.000943,0.000901,2,12,10,2016,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-03,146.899994,147.970001,144.460007,145.380005,143.753632,88570300.0,1,5.056709,0.026927,3.509995,...,0.001747,-0.010672,-0.055716,-0.027821,-0.027186,4,3,6,2022,22
2021-09-10,155.000000,155.479996,148.699997,148.970001,146.686111,140893200.0,1,1.357375,0.007576,6.779999,...,0.007185,0.010257,0.020773,0.001511,0.012601,4,10,9,2021,36
2020-04-28,71.269997,71.457497,69.550003,69.644997,67.832748,112004800.0,1,0.976380,0.014094,1.907494,...,-0.009127,0.025595,0.028538,0.043970,-0.002746,1,28,4,2020,18


In [12]:
import numpy

from itertools import count, takewhile
def frange(start, stop, step):
    return takewhile(lambda x: x< stop, count(start, step))

# list(frange(0, 1, 0.1))
numpy.linspace(5, 10, num=6).tolist()

[5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

In [13]:
# model = xgb.XGBRegressor(n_estimators=100, max_depth=7, eta=0.1, subsample=1, colsample_bytree=.3)
# model = xgb.XGBRegressor(n_estimators=150, max_depth=7, eta=0.05, subsample=1, colsample_bytree=.3)
# model = xgb.XGBRegressor(n_estimators=100, max_depth=7, eta=0.05, subsample=1, colsampvle_bytree=.3)
model = xgb.XGBRegressor(n_estimators=500, max_depth=7, eta=0.05, subsample=0.9, colsample_bytree=0.4)
model.fit(x_train, y_train)

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV, HalvingRandomSearchCV

tune_params = {
    # "n_estimators": [100, 150],
    # "max_depth": [5, 6,],
    # "learning_rate": [0.05, 0.1],

    "n_estimators": [400, 450, 500, 550, 600],
    "max_depth": [6, 7, 8, 9, 11],
    "learning_rate": [0.05, 0.1, 0.2, 0.3, 0.4,],
    "subsample": [0.7, 0.8, 0.9, 1.0, 1.1],
    "colsample_bytree": [0.1, 0.2, 0.3, 0.4, 0.5],

}

# {'subsample': 0.9, 'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.3}
# {'subsample': 0.8, 'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.2}

# xgb_model = xgb.XGBRegressor()
# # model = GridSearchCV(estimator=xgb_model, param_grid=tune_params)
# model = HalvingRandomSearchCV(estimator=xgb_model, param_distributions=tune_params)
# model.fit(x_train, y_train)

# print(model.best_params_)

pickle.dump(model, open("../model/xgboost_model.pkl", "wb")) # Save model as Python pickle object
model.save_model("../model/xgboost_model.json")

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [   0    1    2 ... 3991 3992 3993], got [-0.12864696 -0.12355785 -0.09960737 ...  0.11980827  0.13157265
  0.13904945]

In [ ]:
# print(model.best_params_)
# model


In [ ]:
predicted = model.predict(x_test)
y_test["predicted"] = predicted
y_test

,close_percent_change_1,predicted
date,,
2017-07-13,0.008594,0.003425
2020-08-05,0.034889,0.010051
2023-01-06,0.004089,0.007213
2018-06-01,0.008358,0.003380
2008-02-22,0.002344,0.006831
...,...,...
2020-04-01,0.016687,0.022088
2023-01-27,-0.020078,0.006672
2023-03-14,0.002621,0.006839


In [ ]:
# Get the root mean squared error (RMSE)
mse = np.mean((y_test["predicted"] - y_test[target]) ** 2)
rmse = np.sqrt(mse)
rmse

0.020639225211425125

In [ ]:
# Get the mean absolute error (MAE)
mae = np.mean(np.abs(y_test["predicted"] - y_test[target]))
mae

0.013733777986632346

In [ ]:
# 0.01455269590461311
# 0.013673832772506729
# 0.012685448862479572

In [ ]:
# rmsle
rmsle = np.sqrt(np.mean(np.log1p(np.abs(y_test["predicted"] - y_test[target])) ** 2))
rmsle

0.020035980373427145

In [ ]:
# Get the mean absolute percentage error (MAPE)
mape = np.mean(np.abs((y_test[target] - y_test["predicted"]) / y_test[target])) * 100
mape

inf

In [ ]:
# Save predicted values
y_test.sort_index().to_parquet("../data/predicted.parquet")